In [0]:
%sql
use catalog fraud_catalog;

In [0]:
from pyspark.sql.functions import *
transaction_schema='transaction_id string,user_id string,card_id string,amount double,merchant string,merchant_country string,transaction_time string,device_id string,channel string'
transaction_df=spark.readStream.format('cloudFiles').option('cloudFiles.format','csv').option('header',True).option('cloudFiles.schemaLocation','abfss://data@storageaccountfda.dfs.core.windows.net/catalog/bronze/schemaLocation').option('cloudFiles.schemaHints',transaction_schema).load('abfss://data@storageaccountfda.dfs.core.windows.net/catalog/landing/transaction_streams')
transaction_df=transaction_df.withColumn('file_name',col('_metadata.file_name'))
transaction_df=transaction_df.withColumn('ingest_time',current_timestamp())

In [0]:
transaction_df.writeStream.format('delta').option('checkpointLocation','abfss://data@storageaccountfda.dfs.core.windows.net/catalog/bronze/checkpointLocation').outputMode('append').option('mergeSchema',True).trigger(availableNow=True).start('abfss://data@storageaccountfda.dfs.core.windows.net/catalog/bronze/transaction_streams/data')

In [0]:
from pyspark.sql.functions import *
customer_schema='user_id string,customer_name string,customer_country string,customer_risk_score string'
customer_df=spark.readStream.format('cloudFiles').option('cloudFiles.format','csv').option('header',True).option('cloudFiles.schemaLocation','abfss://data@storageaccountfda.dfs.core.windows.net/catalog/bronze/customers/schemaLocation').option('cloudFiles.schemaHints',customer_schema).load('abfss://data@storageaccountfda.dfs.core.windows.net/catalog/landing/customers')
customer_df=customer_df.withColumn('file_name',col('_metadata.file_name'))\
           .withColumn('ingest_time',current_timestamp())

In [0]:
customer_df.writeStream.format('delta').outputMode('append').option('checkpointLocation','abfss://data@storageaccountfda.dfs.core.windows.net/catalog/bronze/customers/checkpointLocation').option('mergeSchema',True).trigger(availableNow=True).start('abfss://data@storageaccountfda.dfs.core.windows.net/catalog/bronze/customers')

In [0]:
from pyspark.sql.functions import *
country_risk_schema='country_code string,country_namestring,country_risk string'
country_risk_df=spark.readStream.format('cloudFiles').option('cloudFiles.format','csv').option('header',True).option('cloudFiles.schemaLocation','abfss://data@storageaccountfda.dfs.core.windows.net/catalog/bronze/country_risk/schemaLocation').option('schemaHints',country_risk_schema).load('abfss://data@storageaccountfda.dfs.core.windows.net/catalog/landing/country_risks')
country_risk_df=country_risk_df.withColumn('file_name',col('_metadata.file_name'))\
              .withColumn('ingest_time',current_timestamp())

In [0]:
country_risk_df.writeStream.format('delta').outputMode('append').option('checkpointLocation','abfss://data@storageaccountfda.dfs.core.windows.net/catalog/bronze/country_risk/checkpointLocation').trigger(availableNow=True).option('mergeSchema',True).start('abfss://data@storageaccountfda.dfs.core.windows.net/catalog/bronze/country_risk')

In [0]:
from pyspark.sql.functions import *
card_schema='card_id string,user_id string,card_type string,card_status string,issue_date string'
card_df=spark.readStream.format('cloudFiles').option('cloudFiles.format','csv').option('header',True).option('cloudFiles.schemaLocation','abfss://data@storageaccountfda.dfs.core.windows.net/catalog/bronze/cards/schemaLocation').option('schemaHints',card_schema).load('abfss://data@storageaccountfda.dfs.core.windows.net/catalog/landing/cards')
card_df=card_df.withColumn('file_name',col('_metadata.file_name'))\
            .withColumn('ingest_time',current_timestamp())

In [0]:
card_df.writeStream.format('delta').outputMode('append').option('checkpointLocation','abfss://data@storageaccountfda.dfs.core.windows.net/catalog/bronze/cards/checkpointLocation').trigger(availableNow=True).option('mergeSchema',True).start('abfss://data@storageaccountfda.dfs.core.windows.net/catalog/bronze/cards')